In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]=""

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4080094264009605957
]


In [2]:
import numpy as np
import tensorflow as tf

import gensim
import keras as k
import chakin

from keras.preprocessing.text import text_to_word_sequence, one_hot, Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense
from keras.callbacks import ModelCheckpoint, CSVLogger

import util as u

paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
Using TensorFlow backend.


### Initialize Slacker Object

In [3]:
slack = u.initiate_slacker()

Connected to anneke@iitml.


### Load pre-trained word Fast Text

In [4]:
chakin.search(lang='English')

                   Name  Dimension                     Corpus VocabularySize  \
2          fastText(en)        300                  Wikipedia           2.5M   
11         GloVe.6B.50d         50  Wikipedia+Gigaword 5 (6B)           400K   
12        GloVe.6B.100d        100  Wikipedia+Gigaword 5 (6B)           400K   
13        GloVe.6B.200d        200  Wikipedia+Gigaword 5 (6B)           400K   
14        GloVe.6B.300d        300  Wikipedia+Gigaword 5 (6B)           400K   
15       GloVe.42B.300d        300          Common Crawl(42B)           1.9M   
16      GloVe.840B.300d        300         Common Crawl(840B)           2.2M   
17    GloVe.Twitter.25d         25               Twitter(27B)           1.2M   
18    GloVe.Twitter.50d         50               Twitter(27B)           1.2M   
19   GloVe.Twitter.100d        100               Twitter(27B)           1.2M   
20   GloVe.Twitter.200d        200               Twitter(27B)           1.2M   
21  word2vec.GoogleNews        300      

In [5]:
# chakin.download(number=2, save_dir='../../data/')

In [6]:
w2v_model = gensim.models.KeyedVectors.load_word2vec_format(r'../../data/cc.en.300.vec')

### Load data

In [7]:
X_train_sequence = u.open_pickle('../../data/imdb/X_tr_sample_original.pkl')
# X_test_sequence = u.open_pickle('../../data/imdb/X_te_sample_original.pkl')
y_train_target = u.open_pickle('../../data/imdb/y_tr_target_original.pkl')
# y_test_target = u.open_pickle('../../data/imdb/y_te_target_original.pkl')

In [8]:
y_train_target = [' '.join(['UNK', y]) for y in y_train_target]
# y_test_target = [' '.join(['UNK', y]) for y in y_test_target]

Initialize constant here

In [9]:
MAX_ENCODER_SEQ_LEN = 81
MAX_DECODER_SEQ_LEN = 6 #include <UNK>
EMBEDDING_DIM = w2v_model.vector_size
LATENT_DIM = 100

In [10]:
# Tokenize the train sequence data
tokenizer = k.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X_train_sequence)

# Generate text to integer sequence with post padding
X_tr_padded = pad_sequences(tokenizer.texts_to_sequences(X_train_sequence), maxlen=MAX_ENCODER_SEQ_LEN, padding='post', truncating='post')
y_tr_padded = pad_sequences(tokenizer.texts_to_sequences(y_train_target), maxlen=MAX_DECODER_SEQ_LEN, padding='post', truncating='post')

In [11]:
NUM_DECODER_TOKENS = len(tokenizer.word_index)

In [12]:
NUM_DECODER_TOKENS

42406

Initialize the container for input sequence decoder and encoder

In [13]:
encoder_input_data = np.zeros(
    (len(X_train_sequence), MAX_ENCODER_SEQ_LEN, EMBEDDING_DIM),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(y_train_target), MAX_DECODER_SEQ_LEN, EMBEDDING_DIM),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(y_train_target), MAX_DECODER_SEQ_LEN, len(tokenizer.word_index)),
    dtype='float32')

In [14]:
index_to_word = {v: k for k, v in tokenizer.word_index.items()}
index_to_word[0] = ' '

In [15]:
for i, (input_sequence, target_sequence) in enumerate(zip(X_tr_padded, y_tr_padded)):
    # embed the input sequence
    for t, index in enumerate(input_sequence):
        try:
            encoder_input_data[i, t, :] = w2v_model[index_to_word[index]]
        except KeyError as error:
            pass
    
    # embed the input decoder
    for t, index in enumerate(target_sequence):
        try:
            decoder_input_data[i, t, :] = w2v_model[index_to_word[index]]
        except KeyError as error:
            pass
        
    for t, index in enumerate(target_sequence):
        # not include the first <UNK>
        if t>0:
            decoder_target_data[i, t - 1, index] = 1.

# Model callbacks

In [17]:
# checkpoint = k.callbacks.ModelCheckpoint(save_best_only=True, monitor='val_loss', filepath='./300_fasttext_best_model/weights.{epoch:04d}-{val_loss:.3f}.h5')
# csvlogger = k.callbacks.CSVLogger(filename='fasttext_300_history.log', append=True)

### Train

In [18]:
# encoder_inputs = k.layers.Input(shape=(None, EMBEDDING_DIM))
# encoder = LSTM(LATENT_DIM, return_state=True)
# encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# encoder_states = [state_h, state_c]

In [19]:
# decoder_inputs = k.layers.Input(shape=(None, EMBEDDING_DIM))
# decoder_lstm = LSTM(LATENT_DIM, return_sequences=True, return_state=True)
# decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
#                                     initial_state=encoder_states)

# decoder_dense = k.layers.Dense(NUM_DECODER_TOKENS, activation='softmax')
# decoder_outputs = decoder_dense(decoder_outputs)

In [20]:
# train_model = k.models.Model([encoder_inputs, decoder_inputs], decoder_outputs)
# train_model.compile(optimizer='adam', loss='categorical_crossentropy')

In [21]:
epochs=200
batch_size=256

In [22]:
# train_model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
#           batch_size=batch_size,
#           epochs=epochs,
#           validation_split=(1./3),
#           verbose=1, callbacks=[checkpoint, csvlogger])

Train on 15168 samples, validate on 7584 samples
Epoch 1/200
15168/15168 [==============================] - 48s 3ms/step - loss: 6.5550 - val_loss: 4.3574


/home/anneke/.local/lib/python3.6/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 100) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 100) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


Epoch 2/200
15168/15168 [==============================] - 42s 3ms/step - loss: 4.1436 - val_loss: 4.1754
Epoch 3/200
15168/15168 [==============================] - 42s 3ms/step - loss: 4.0138 - val_loss: 4.1091
Epoch 4/200
15168/15168 [==============================] - 42s 3ms/step - loss: 3.9402 - val_loss: 4.0683
Epoch 5/200
15168/15168 [==============================] - 42s 3ms/step - loss: 3.8889 - val_loss: 4.0402
Epoch 6/200
15168/15168 [==============================] - 42s 3ms/step - loss: 3.8496 - val_loss: 4.0138
Epoch 7/200
15168/15168 [==============================] - 42s 3ms/step - loss: 3.8163 - val_loss: 3.9900
Epoch 8/200
15168/15168 [==============================] - 42s 3ms/step - loss: 3.7802 - val_loss: 3.9624
Epoch 9/200
15168/15168 [==============================] - 42s 3ms/step - loss: 3.7340 - val_loss: 3.9115
Epoch 10/200
15168/15168 [==============================] - 42s 3ms/step - loss: 3.6865 - val_loss: 3.8657
Epoch 11/200
15168/15168 [===================

15168/15168 [==============================] - 42s 3ms/step - loss: 1.7407 - val_loss: 2.8996
Epoch 155/200
15168/15168 [==============================] - 42s 3ms/step - loss: 1.7318 - val_loss: 2.8959
Epoch 156/200
15168/15168 [==============================] - 42s 3ms/step - loss: 1.7267 - val_loss: 2.8988
Epoch 157/200
15168/15168 [==============================] - 42s 3ms/step - loss: 1.7162 - val_loss: 2.8951
Epoch 158/200
15168/15168 [==============================] - 42s 3ms/step - loss: 1.7124 - val_loss: 2.8965
Epoch 159/200
15168/15168 [==============================] - 42s 3ms/step - loss: 1.7036 - val_loss: 2.8997
Epoch 160/200
15168/15168 [==============================] - 42s 3ms/step - loss: 1.6903 - val_loss: 2.9169
Epoch 161/200
15168/15168 [==============================] - 42s 3ms/step - loss: 1.6852 - val_loss: 2.8981
Epoch 162/200
15168/15168 [==============================] - 42s 3ms/step - loss: 1.6742 - val_loss: 2.9005
Epoch 163/200
15168/15168 [===============

In [16]:
model = load_model('./300_fasttext_best_model/weights.0168-2.893.h5')

In [17]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 300)    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 300)    0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 100), (None, 160400      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 100),  160400      input_2[0][0]                    
                                                                 lstm_1[0][1]                     
          

### Inference on Train/test set

In [19]:
model = load_model('./300_fasttext_best_model/weights.0168-2.893.h5')

encoder_inputs = model.input[0]
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]
decoder_state_input_h = Input(shape=(LATENT_DIM,), name='input_5')
decoder_state_input_c = Input(shape=(LATENT_DIM,), name='input_6')

decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_lstm = model.layers[3]

decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
decoder_inputs, initial_state=decoder_states_inputs)

decoder_states = [state_h_dec, state_c_dec]

decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
                    [decoder_inputs] + decoder_states_inputs,
                    [decoder_outputs] + decoder_states
                    )



In [20]:
word_index = tokenizer.word_index
reverse_word_index = dict((i,word) for word,i in word_index.items())

In [21]:
def generate_candidate_list(X):
    y_candidate = []
    
    for i in range(X.shape[0]-1-5):
        y_candidate.append(X[i:i+5])
    
    return np.asarray(y_candidate)

def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 

def target_index(doc_idx, candidate_seq, y):
    for i,j in enumerate(candidate_seq):
        if len(intersection(j, y)) == len(y):
            return i
    return -1

# doc num, doc index argmax

def to_sequence(int_sequence):
    decoded = ''
    for i,intnum in enumerate(int_sequence):
        if intnum == 0:
            word = '<PAD>'
        else:
            word = reverse_word_index[intnum]
        
        if i == len(int_sequence):
            decoded += word
        else:
            decoded += word + ' '
    return decoded

def rouge_one(true, candidate, start_index):
    
    if isinstance(true, str) and isinstance(candidate, str):
        true = true.split()
        candidate = candidate.split()
    
    overlap = [value for value in true[start_index:] if value in candidate[start_index:]] 

    
    if len(true[start_index:]) != 0:
        recall = len(overlap)/len(true[start_index:])
    else:
        recall = 0
    
    if len(candidate[start_index:]):
        precision = len(overlap)/len(candidate[start_index:])
    else:
        precision = 0
    
    if (recall+precision) != 0:    
        f1 = 2*((recall*precision)/(recall+precision))
    else:
        f1 = 0
    
    return recall, precision, f1

In [22]:
# Play with candidate

def decode_sequence_target(candidate_states_value, candidate_target_seq):
#     candidate_states_value = encoder_model.predict(input_seq)

    from_candidate_target_seq = np.zeros((1,1, EMBEDDING_DIM))
    
    candidate_token_index = candidate_target_seq[0,0]
    try:
        from_candidate_target_seq[0,0,:] = w2v_model[index_to_word[candidate_token_index]]
    except KeyError as error:
        pass
    
    candidate_joint_log_prob = 0
    
    for i in range(1,5):
        from_candidate_output_tokens, h_true, c_true = decoder_model.predict([from_candidate_target_seq] + candidate_states_value)
    
        candidate_target_prob = from_candidate_output_tokens[0,-1, candidate_target_seq[0,i]]
        candidate_joint_log_prob += np.log(candidate_target_prob)
        
        # get the t+1 input
        
        candidate_token_index = candidate_target_seq[0,i]
        from_candidate_target_seq = np.zeros((1,1,EMBEDDING_DIM))
        try:
            from_candidate_target_seq[0,0,:] = w2v_model[index_to_word[candidate_token_index]]
        except KeyError as error:
            pass
        
        
        candidate_states_value = [h_true, c_true]

    return candidate_joint_log_prob, candidate_target_prob

In [23]:
i=0
file = open("candidate_jll_300_fasttext_imdb.csv", "w")

In [24]:
for doc in X_tr_padded:
    y_candidate = generate_candidate_list(doc)
    
    candidate_jll_per_doc = []
    input_seq = encoder_input_data[i:i+1]
    
    true_target_index = target_index(i, y_candidate, y_tr_padded[i])
#     print(y_candidate)
    # Encode
    candidate_states_value = encoder_model.predict(input_seq)
    
    for j in range(y_candidate.shape[0]):
        candidate_seq = y_candidate[j:j+1]
        candidate_jll_slide, candidate_last_prob = decode_sequence_target(candidate_states_value, candidate_seq)
        candidate_jll_per_doc.append(candidate_jll_slide)

    candidate_jll_per_doc = np.asarray(candidate_jll_per_doc)
    max_jll_index = np.argmax(candidate_jll_per_doc)
    true_target_jll = np.around(candidate_jll_per_doc[true_target_index],5)
    max_candidate_jll = np.around(candidate_jll_per_doc[max_jll_index],5)
    
    # get recall here
    [precision, recall, f_score] = rouge_one(y_train_target[i], to_sequence(y_candidate[max_jll_index]), 1)
    
    file.write('%d\t%d\t%s\t%d\t%s\t%d\t%.5f\t%.5f\t%.5f\t%d\t%.5f\t%.5f\t%.5f\t%.5f\t%.5f\n' %(i, true_target_index, y_train_target[i],
                                                            max_jll_index, to_sequence(y_candidate[max_jll_index]),
                                                            -(true_target_index-max_jll_index),
                                                            true_target_jll, max_candidate_jll,
                                                            np.absolute(true_target_jll-max_candidate_jll),
                                                            len(intersection(y_tr_padded[i], y_candidate[max_jll_index])),
                                                            np.exp(true_target_jll/4), np.exp(max_candidate_jll/4),
                                                            precision, recall, f_score))
    
#     print('%d\t%d\t%s\t%d\t%s\t%d\t%.5f\t%.5f\t%.5f\t%d\n' %(i, true_target_index, y['text'][i],
#                                                             max_jll_index, to_sequence(y_candidate[max_jll_index]),
#                                                             -(true_target_index-max_jll_index),
#                                                             true_target_jll, max_candidate_jll,
#                                                             np.absolute(true_target_jll-max_candidate_jll),
#                                                             len(intersection(y['padded'][i], y_candidate[max_jll_index]))))

#     print('%s\t%s\t%.1f\n' %(y_train_target[i], to_sequence(y_candidate[max_jll_index]), precision, recall))
    if i % 1000 == 0:
#         print('Processing document %d...' %(i))
        msg = 'glove 100: processing document ' + str(i)
        u.slack_post_message(slack, msg, 'deep-learning', 'test')
        print(msg)
        
    i += 1
    
file.close()
report_stats('Processing DONE', 'deep-learning')

glove 100: processing document 0
glove 100: processing document 1000
glove 100: processing document 2000
glove 100: processing document 3000
glove 100: processing document 4000
glove 100: processing document 5000
glove 100: processing document 6000
glove 100: processing document 7000
glove 100: processing document 8000
glove 100: processing document 9000
glove 100: processing document 10000
glove 100: processing document 11000
glove 100: processing document 12000
glove 100: processing document 13000
glove 100: processing document 14000
glove 100: processing document 15000
glove 100: processing document 16000
glove 100: processing document 17000
glove 100: processing document 18000
glove 100: processing document 19000
glove 100: processing document 20000
glove 100: processing document 21000
glove 100: processing document 22000


IndexError: index 42406 is out of bounds for axis 2 with size 42406